In [1]:
import cv2
import numpy as np
#from google.colab.patches import cv2_imshow
#import math
#import time
from IPython import display as display

import ipywidgets as ipw
import PIL
from io import BytesIO


In [37]:
class Particle:
  MaxV = np.sqrt(2)
  def __init__(self,x,y,vx,vy,WallParticle=False):
    self.r = np.array([float(x),float(y)])
    self.v = np.array([float(vx),float(vy)])
    self.WallParticle=WallParticle
    self.F=np.array([0.0,0.0])
  def normalize_vector(self,x):
    norm=np.linalg.norm(x)
    if norm==0:
      return x*np.inf
    return x/norm
  def r12(self,r2):
    return np.linalg.norm(r2-self.r)
  def calculateF(self,r2):
    if self.WallParticle==True:
      return np.array([0.,0.])
    r12=self.r-r2
    r12magnitude=np.linalg.norm(r12)
    if r12magnitude<=2:
      return self.normalize_vector(r12)/ (r12magnitude**2)
    return np.array([0.,0.])
  def update_r(self):
    if self.WallParticle==True:
      return
    self.r+=self.v
    return
  def update_v(self):
    if self.WallParticle==True:
      return
    self.v+=self.F

    vmag=np.linalg.norm(self.v)

    if vmag> self.MaxV:
      self.v=self.normalize_vector(self.v)*self.MaxV
    return
    
  def graph(self,x0,y0,img):
    if self.WallParticle==True:
      color=(0,255,255)
    else:
      color=(255,255,255)

    cv2.circle(img,(int(x0+self.r[0]),int(y0-self.r[1])),10,color,-1)
    return




In [39]:
particles=[]

def lineOfWallParticles(x1,y1,x2,y2,N):
  global particles
  x=np.linspace(x1,x2,N)
  y=np.linspace(y1,y2,N)
  for i in range(N):
    particles.append(Particle(x[i],y[i],0,0,WallParticle=True))



wIm = ipw.Image()
display.display(wIm)


maxX=500
maxY=500
x0=int(maxX/2)
y0=int(maxY/2)

  
img = np.zeros((500, 500, 3), dtype="uint8")

lineOfWallParticles(-100,100,100,100,20)
lineOfWallParticles(-100,-100,100,-100,20)
lineOfWallParticles(-100,100,-100,-100,20)
lineOfWallParticles(100,-100,100,100,20)

particles.append(Particle(0,0,0.5,0.5))
particles.append(Particle(50,50,0.5,0.5))

MaxIterations = 2000

NumParticles=len(particles)

for count in range(MaxIterations):
  img[:]=(0,0,0)
  for i in range(NumParticles):
    for j in range(NumParticles):
      if i!=j:
        Fij=particles[i].calculateF(particles[j].r)
        particles[i].F+=Fij
  for p in particles:
    p.update_v()
    p.update_r()
    p.graph(x0,y0,img)
    p.F[:]=0
  
  cv2.putText( img,str(count),(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255))
  
  pilIm = PIL.Image.fromarray(img, mode="RGB")
  with BytesIO() as fOut:
      pilIm.save(fOut, format="png")
      byPng = fOut.getvalue()
        
  # set the png bytes as the image value; 
  # this updates the image in the browser.
  wIm.value=byPng  




Image(value=b'')

KeyboardInterrupt: ignored